## 1 - Import

In [37]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse
import fitz


In [5]:
load_dotenv()

True

## 2 - Models

In [6]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [7]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=1000
)


/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_15398/426928708.py:1: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


## 3 - Chunking

### 3.1 - data input

In [8]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [9]:
url_aml_5_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32015L0849"
url_crr_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32013R0575"

### 3.2 - Url to local pdf

In [10]:
def url_to_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, "wb") as f:
        f.write(response.content)

In [11]:
url_to_pdf(url_aml_5_pdf,"/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

### 3.3 - PDFloader (option 1)

In [12]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
def pdf_loader(path):
    loader = PyPDFLoader(file_path=path, 
                         mode='single') # returned as a single langchain Document object
                                        # alternative to try: use “elements” mode, the unstructured 
                                        # library will split the document into elements such as Title and 
                                        # NarrativeText
    doc= loader.load()
    return doc

In [16]:
url_crr_pdf_txt= pdf_loader("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

In [35]:
url_crr_pdf_txt[0].page_content

"DIRECTIVES \nDIRECTIVE (UE) 2015/849 DU P ARLEMENT EUR OPÉEN ET DU CONSEIL \ndu 20 mai 2015 \nrelativ e à la prév ention de l'utilisation du sys tème f inancier aux f ins du blanchiment de capita ux \nou du f inancement du ter ror isme, modif iant le règlement (UE) n\no \n648/2012 du Parlement européen \net du Conseil et abrogeant la directiv e 2005/60/CE du Parlement européen et du Conseil et la \ndirectiv e 2006/70/CE de la Commission \n(T exte présentant de l'intérêt pour l'EEE) \nLE P ARLEMENT EUR OPÉEN ET LE CON SEIL DE L'UNION EUR OPÉENNE, \nvu le traité sur le f onctionnement de l'Union européenne, et notamment son ar ticle 114, \nvu la proposition de la Commission européenne, \naprès transmission du projet d'acte législatif aux parlements nationaux, \nvu l'avis de la Banque centrale européenne (\n1\n), \nvu l'avis du Comit é économique et social européen (\n2\n), \nstatuant conf or mément à la procédure législative ordinaire (\n3\n), \nconsidérant ce qui suit: \n(1)  Les f lux

### 3.3 - fitz (option 2)

## 3 - Chunking

In [14]:
article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
# attention spliter	Si le « morceau » résultant dépasse chunk_size, il redescend au séparateur suivant (plus petit).
    chunk_overlap=0)


In [15]:
chunks_aml5 = article_splitter.split_text(document_aml_pdf_step_4)
chunks_aml5

NameError: name 'document_aml_pdf_step_4' is not defined

## 4 - Après

In [ ]:
def load_txt(filepath: str):
    """
    Charge un fichier txt dans une variable str
    Args: filepath (str)
    Returns: str
    
    """
    with open(filepath, "r", encoding="utf-8") as file:
        return file.read()
    
   
def save_txt(filepath: str, text: str):
    """
    Sauvegarde le texte donné dans un fichier .txt.
    
    Args:
        filepath (str): Chemin du fichier où sauvegarder le texte.
        text (str): Contenu texte à écrire.
    """
    with open(filepath, "w", encoding="utf-8") as fichier:
        fichier.write(text)

In [ ]:
fr_old_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [ ]:
fr_new_version= "Les entreprises établies en France et qui font partie d'un groupe financier ou d'un groupe comprenant au moins une société de financement ou, pour l'application du 2° du présent article, d'un groupe au sens de l'article L. 356-1 du code des assurances, d'un groupe au sens du III de l'article L. 511-20 comprenant au moins une entité mentionnée au 1° bis ou 1° ter de l'article L. 561-2, ou d'un groupe mixte ou d'un conglomérat financier auquel appartiennent des entités réglementées au sens de l'article L. 517-2 ayant leur siège social dans un Etat membre de l'Union européenne ou Etat partie à l'accord sur l'Espace économique européen ou dans un Etat où sont applicables les accords prévus aux articles L. 632-7 , L. 632-13 et L. 632-16 du présent code sont tenues, nonobstant toutes dispositions contraires, de transmettre à des entreprises du même groupe ayant leur siège social dans l'un de ces Etats : 1° Les renseignements relatifs à leur situation financière nécessaires à l'organisation de la surveillance sur base consolidée et de la surveillance complémentaire de ces entités réglementées ou sociétés de financement ; 2° Les informations nécessaires à l'organisation de la lutte contre le blanchiment des capitaux et contre le financement du terrorisme ; 3° Les informations nécessaires à l'organisation de la détection des abus de marché mentionnés à l'article 16 du règlement (UE) n° 596/2014 du Parlement européen et du Conseil du 16 avril 2014 sur les abus de marché (règlement relatif aux abus de marché) et abrogeant la directive 2003/6/CE du Parlement européen et du Conseil et les directives 2003/124/CE, 2003/125/CE et 2004/72/CE de la Commission ; 4° Les informations nécessaires à la gestion des conflits d'intérêts au sens du 3° du I et 3° du II de l'article L. 533-10 . Ces dernières informations ne peuvent être communiquées à des personnes extérieures au groupe, à l'exception des autorités compétentes des Etats visés au premier alinéa. Cette exception ne s'étend pas aux autorités des Etats ou territoires dont la législation est reconnue insuffisante ou dont les pratiques sont considérées comme faisant obstacle à la lutte contre le blanchiment des capitaux ou le financement du terrorisme par l'instance internationale de concertation et de coordination en matière de lutte contre le blanchiment d'argent dont la liste est mise à jour par arrêté du ministre chargé de l'économie. Les personnes recevant ces informations sont tenues au secret professionnel dans les conditions et sous les peines mentionnées à l'article L. 511-33 , pour tous renseignements ou documents qu'elles seraient ainsi amenées à recevoir ou à détenir. Les dispositions du présent article ne font pas obstacle à l'application de la loi n° 78-17 du 6 janvier 1978 relative à l'informatique, aux fichiers et aux libertés."

In [ ]:
scrape_result= load_txt("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/aml5.txt")

In [ ]:
def get_eu_data_3p(directive_UE_CRR_step4, fr_old_version, fr_new_version):
    prompt_LCQA_3p = f""" 
        Tu es un juriste expert avec au moins 10 ans d expérience.
        Tu as une double expertise a la fois dans la réglementation francaise et dans les textes européens.
        Tu as travaillé a plusieurs reprises sur la transposition des textes européens dans la réglementation francaise.
        
        Ta tâche :
        -   Analyser les changements réglementaires dans la loi francaise avec pour objectif de trouver la ou les parties
        de la directive UE (article ou articles) a l origine de ce(s) changement(s).
        
        Régles : 
        - Tu te baseras uniquement sur les informations fournies pour faire ton analyse.
        - Tu feras ton analyse de la facon la plus conconcieuse et précise 
        - Tu ne pas pas reciter le contenu de la loi francaise avant et après modification
        
        Voici le text européen {scrape_result}
        Voici le texte de loi francais dans sa version avant l'entree en vigueur du text européen: {fr_old_version}
        Voici le texte de loi francais dans sa version après l'entree en vigueur du text européen: {fr_new_version}
"""
    output = llm_4omini.invoke(prompt_LCQA_3p)
    output= output.content
    return output

In [ ]:
get_eu_data_3p(scrape_result, fr_old_version,fr_new_version)

"L'analyse des modifications apportées à la législation française suite à l'entrée en vigueur de la directive (UE) 2015/849 révèle plusieurs éléments clés qui correspondent à des articles spécifiques de la directive.\n\n1. **Extension du champ d'application** : La mention des groupes financiers et des entités réglementées a été élargie pour inclure des entités mentionnées au 1° bis ou 1° ter de l'article L. 561-2. Cela fait écho à l'article 2 de la directive, qui définit les entités assujetties et leur champ d'application.\n\n2. **Obligations de vigilance** : Les obligations de vigilance à l'égard de la clientèle, notamment en ce qui concerne la transmission d'informations pour la lutte contre le blanchiment de capitaux et le financement du terrorisme, sont renforcées. Cela se rapporte à l'article 11 de la directive, qui impose des mesures de vigilance à l'égard de la clientèle dans divers cas.\n\n3. **Secret professionnel** : La protection des informations transmises et le respect du 